## Definição do *dataset*

O *dataset* utilizado será o "Detecting Individual and Combined Fingers Movements" [1]. Maiores informações podem ser vistas no site: https://www.rami-khushaba.com/electromyogram-emg-repository.html

De acordo com a figura seguinte, neste *dataset* existem 15 movimentos. Algumas questões de projetos foram levadas em consideração:
1. Cada 

[1] R. N. Khushaba, Maen Takruri, Jaime Valls Miro, and Sarath Kodagoda, "Towards limb position invariant myoelectric pattern recognition using time-dependent spectral features", Neural Networks, vol. 55, pp. 42-58, 2014. https://doi.org/10.1016/j.neunet.2014.03.010


In [1]:
# Dependências
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import math
from librosa import stft
from sklearn import svm

In [2]:
# Obtendo dados
data = np.array([[genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/HC_1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/HC_2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/HC_3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_I1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_I2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_I3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_M1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_M2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/I_M3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/IMR1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/IMR2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/IMR3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/L_L1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/L_L2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/L_L3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_M1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_M2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_M3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_R1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_R2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/M_R3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/MRL1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/MRL2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/MRL3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_L1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_L2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_L3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_R1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_R2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/R_R3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_I1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_I2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_I3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_L1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_L2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_L3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_M1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_M2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_M3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_R1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_R2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_R3.csv', delimiter=',', unpack=True)],
                [genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_T1.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_T2.csv', delimiter=',', unpack=True),
                 genfromtxt('./Delsys_8Chans_15Classes/S1-Delsys-15Class/T_T3.csv', delimiter=',', unpack=True)]]
               )

print(data.shape)

(15, 3, 8, 80000)


In [3]:
# Segmentando os dados
# Definição do salto e do tamanho do segmento (segmento - salto = sobreposição)
salto = 600
segmento = 800
n_win = int((20000 - segmento) / salto) + 1
ids = np.arange(n_win) * salto
x = np.array([data[:,:,:,k:(k + segmento)] for k in ids]).transpose(1, 2, 3, 0, 4)
print(x.shape)

(15, 3, 8, 33, 800)


In [4]:
# Domínio do tempo

In [5]:
# Calculando o MAV
print(x.shape)
mav = np.sum(abs(x)/800, axis=-1)
print(mav.shape)

(15, 3, 8, 33, 800)
(15, 3, 8, 33)


In [6]:
# Calculando VAR
print(x.shape)
var = np.sum(np.power(x, 2)/799, axis=-1)
print(var.shape)

(15, 3, 8, 33, 800)
(15, 3, 8, 33)


In [7]:
# Calculando SSI
print(x.shape)
ssi = np.sum(np.power(abs(x), 2), axis=-1)
print(ssi.shape)

(15, 3, 8, 33, 800)
(15, 3, 8, 33)


In [8]:
# Calculando RMS
print(x.shape)
rms = np.sqrt(np.sum((np.power(abs(x), 2))/800, axis=-1))
print(rms.shape)

(15, 3, 8, 33, 800)
(15, 3, 8, 33)


In [9]:
# Domínio da frequência

In [21]:
# Calculando FFT
# shape: [15 movimentos, 3 trials, 8 eletrodos, 20000 samples]
# Aplicando stft no último eixo de data (3), com janela de 800 e sobreposição de 600
w = np.apply_along_axis(stft, 3, data[:,:,:,0:19200], n_fft=800, hop_length=600)
w = np.swapaxes(w, 3, 4)
print(w.shape)

(15, 3, 8, 33, 401)


In [22]:
# Calculando PSD
psd = np.power(abs(w), 2)
print(psd.shape)

(15, 3, 8, 33, 401)


In [23]:
# Calculando FMD
fmd = np.sum(psd/2, axis=-1)
print(fmd.shape)

(15, 3, 8, 33)


In [24]:
# Calculando FMN

sampleRate = 4000
M = 401
f = np.array([(i*sampleRate)/(2*M) for i in range(1,M+1)])

fmn = np.divide((np.sum(np.multiply(psd,f), axis = -1)), (np.sum(psd, axis=-1)))

print(fmn.shape)

(15, 3, 8, 33)


In [52]:
# Juntando características
features = np.array([mav, var, ssi, rms, fmd, fmn])
print("(6 características, 15 movimentos, 3 tentativas, 8 eletrodos, 33 segmentos)")
print(features.shape)

# Seperando treinamento
featuresTraining = features[:,:,0,:,:].transpose(1,3,0,2)
featuresTraining = np.concatenate(featuresTraining).transpose(2,1,0)
print(featuresTraining.shape)

(6 características, 15 movimentos, 3 tentativas, 8 eletrodos, 33 segmentos)
(6, 15, 3, 8, 33)
(8, 6, 495)


In [27]:
# Aplicando a media nas trials
features = np.array([mav, var, ssi, rms, fmd, fmn])
print(features.shape)
features = features[:,:,0:2,:].transpose(0,2,1,3,4)
#  # capturando as 2 trials
# print(features.shape)

# # Concatenando características
# concatData = list()
# for i in range(4):
#     concatData.append(features[i,:,:,:].reshape(-1))

# concatData = np.asarray(concatData).transpose(1,0)
# print(concatData.shape)

(6, 15, 3, 8, 33)


In [17]:
# Classificando

# Aplicando SVM
X = concatData
y = list()

for k in range(2):
    j = 1
    for i in range(1,3961):
        y.append(j)
        if((i%264)==0):
            j+=1

#print((y))
clf = svm.SVC(gamma='auto')
clf.fit(X, y)
print(clf.predict(concatData[0:258,:]))

[12  8 14 14 14  3  3  3  3  3  3 14  3  3  6  3  5  3  3  3 11  3  6  6
  3  6  6  5  6  6  6  6  6  2  2  6  6  6  6  6  2  2  2  2  6  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15  2  2 15 15 15 15 15 15 15 15 15 15 15  2  2 15 15 15 15 15 15
 15 15 15 15 15 15  2 15 15 15 15  2  8 12 12 12 12 12  8 14 14 14 14 14
 14 14 14 14  3  3 14  3  3  3  3  3  3  3  3 14 14 14  3 14 14 14 14 14
 14 14 14 14  3  3 14 14  3  6  3  5  3  6  6  6  6  6  6  6  6  6  6  6
  2  2  2  2  2  6  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12 12  1
  1 12 12 12  1 12  1 12  1 12 12 12  1  1 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12  8  8 12  8 14 14 14 14 14 14 12]
